# Principles of Digital Communication - Project
_Harold Benoit, Tom Demont_

_May 2021_

In [1082]:
import numpy as np

## 1. Text conversion from string to bits
We will first create the utility methods for conversion from utf-8 text.

In [1083]:
def string_to_bitarray(string):
    string_bytes = string.encode('utf-8')
    bits = []
    for b in string_bytes:
        mask = 1
        for i in range(7,-1,-1):
            bits.append((b >> i)&mask)
    return bits

In [1084]:
print(string_to_bitarray('Hello World 😃'))

[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1]


In [1085]:
def bitarray_to_string(bitarray):
    bytes = []
    # the format for putting one bit in a string
    string = "{0:0"+str(1)+"b}"

    # go over all bytes given
    for nb_bytes in range(0, len(bitarray)//8):
        # recreates the byte from the bits
        byte = ''
        for bit in range(0,8):
            byte += string.format(bitarray[nb_bytes*8+bit])
        # adds the integer associated to the binary string in that byte (between 0 and 255)
        bytes.append(int(byte,2))
    return str(bytearray(bytes), 'utf-8')

#### Testing our system
We verify that transforming a string to a bit array back and forth works well:

In [1086]:
bitarray_to_string(string_to_bitarray('Hello World 😃'))

'Hello World 😃'

## 2. Recover from channel erasure
To recover from channel erasure, we'll implement a **parity check**.

### Encoding for channel erasure

Per every 2 uses of the channel, we'll send a third parity check codeword. It is generated from applying xor operator
 on the 2 first encoded blocks bits and sending the associated codeword.

For example, when $k=3$, suppose we want to send $001$ and $010$. $001$ is encoded by $c_1$ and $010$ is encoded by
$c_2$. Our parity codeword will then be $001 \oplus 010=011$, encoded by $c_3$.

In [1087]:
def compute_parity_check(bitarray_1, bitarray_2):
    parity_check = []
    for bit in range(0,len(bitarray_1)):
        parity_check.append(bitarray_1[bit] ^ bitarray_2[bit])
    return parity_check

In [1088]:
print("Hello        ",string_to_bitarray('Hello'))
print("World        ",string_to_bitarray('World'))
print("Parity check ",compute_parity_check(string_to_bitarray('Hello'),string_to_bitarray('World')))

Hello         [0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1]
World         [0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0]
Parity check  [0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1]


Also, we'll interleave our codewords so that, erasing one channel symbol over 3 will in fact erase a full codeword.
We'll recover this one with the parity check by inverting our xor operation.

For example, if we want to send $c_1c_2c_3$, we'll send $c_{10}c_{20}c_{30}c_{11}c_{21}c_{31}...c_{1n}c_{2n}c_{3n}$

In [1089]:
def interleave_codewords_to_send(codeword_1, codeword_2, parity_check_encoded):
    codewords = np.array([codeword_1, codeword_2, parity_check_encoded])
    return codewords.transpose().flatten()

In [1090]:
# Tests interleaving 'Hello', 'World' and their parity check
interleaved_hello_world = interleave_codewords_to_send(string_to_bitarray('Hello'), string_to_bitarray('World'),
                                           compute_parity_check
(string_to_bitarray('Hello'),string_to_bitarray('World')))
print(interleaved_hello_world)

[0 0 0 1 1 0 0 0 0 0 1 1 1 0 1 0 1 1 0 1 1 0 1 1 0 0 0 1 1 0 1 1 0 0 0 0 0
 1 1 1 1 0 0 1 1 1 1 0 0 0 0 1 1 0 1 1 0 0 1 1 1 0 1 1 0 1 0 1 1 0 0 0 0 0
 0 1 1 0 1 1 0 0 0 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0 0 0 0 1 0 1
 1 1 0 1 0 1 1 0 1]


### Making an erasing channel
We build the channel that randomly removes $1/3$ of the coordinates given

In [1091]:
# the channel input is the array of interleaved codewords and their parity check
def erasing_channel(chan_input):
    chan_input = np.clip(chan_input,-1,1)
    erased_index = np.random.randint(3)
    chan_input[erased_index:len(chan_input):3] = 0
    return chan_input

### Build a simple codewords book for erasure testing
The codebook will be as follows: $0 \to -1$ and $1 \to 1$. This codebook is very simple, has $k=n=1$ and will just be
 useful for making our bitstring able to go through the erasing channel.

In [1092]:
def encode(bitarray):
    encoded = []
    for bit in bitarray:
        if bit == 0:
            encoded.append(-1)
        elif bit == 1:
            encoded.append(1)
        else:
            print("bit array should only contain 0 or 1")
            return
    return encoded

In [1093]:
print("Hello binary:  ",string_to_bitarray('Hello'))
print("Hello encoded: ",encode(string_to_bitarray('Hello')))

Hello binary:   [0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1]
Hello encoded:  [-1, 1, -1, -1, 1, -1, -1, -1, -1, 1, 1, -1, -1, 1, -1, 1, -1, 1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, 1, 1, 1]


### Decoding procedure
We'll :
1. Uninterleave the received array to retrieve what should be associated to each codeword
2. Find the codeword that was erased by looking for a $[0,...,0]$ array
3. Decode the codewords that was correctly sent
  * If the erased codeword is the third one, it means that only the parity check was affected which is not a problem,
   we just return the decoded bits from both first codewords
   * If the erased codeword is one of both first, we apply $\oplus$ on both correctly decoded bits to retrieve the
   erased
    one thanks to the parity check

In [1094]:
# the triplet contains 2 codewords encoding k bits and the codeword encoding the parity check bits
def uninterleave_codewords_received(three_codewords):
    # we create chunks of size 3, the interleaved codewords matrix (shape (n,3))
    codewords_triplet = np.array_split(np.array(three_codewords), len(three_codewords)//3)

    # we transpose it to retrieve the aligned codewords in order
    codewords = np.array(codewords_triplet).transpose().flatten()
    return (codewords)

In [1095]:
# gives both correctly decoded codewords and the value of H observed (the modulus class of erased coordinate)
def decode_output_retrieve_erased(uninterleaved_three_codewords):
    decoded = []
    erased = -1
    n = len(uninterleaved_three_codewords)//3

    # classic decoding from our codebook
    for i,symbol in enumerate(uninterleaved_three_codewords):
        if symbol == 1:
            decoded.append(1)
        elif symbol == -1:
            decoded.append(0)
        elif symbol == 0:
            # this one was erased
            erased = (i//n)%3
        else:
            print("Error: symbols cannot be other than 0 and ±1")
            return
    return decoded,erased

In [1096]:
# decoded contains both correctly decoded codewords (can be information codeword and parity or 2 information codewords)
# the parity codeword (present if erased != 2) is in the last half of the decoded bit array
def restore_erased(decoded, erased):
    if erased != 2:
        # an information codeword was erased
        restored = []
        n = len(decoded)//2
        for i in range(n):
            # we recompute the erased one by binary one time padding
            restored.append(decoded[i]^decoded[n+i])
        if erased == 0:
            # the first codeword was erased
            return np.array([restored,decoded[0:len(decoded)//2]]).flatten()
        elif erased == 1:
            # the second codeword was erased
            return np.array([decoded[0:len(decoded)//2],restored]).flatten()
    else:
        # the parity check codeword was erased
        return decoded

#### Testing our system
We now try to send "HelloWorld" message over the erasing channel and try to restore it:

In [1097]:
channel_input = encode(interleaved_hello_world)
print("Channel input:    ", channel_input)
channel_output = erasing_channel(channel_input)
print("Channel output:   ", list(channel_output))

print()

uninterleaved_input = encode(np.array([string_to_bitarray('Hello'), string_to_bitarray('World'),compute_parity_check
(string_to_bitarray('Hello'),string_to_bitarray('World'))]).flatten())
print("Uninterleaved in: ", list(uninterleaved_input))
uninterleaved_output = uninterleave_codewords_received(channel_output)
print("Uninterleaved out:", list(uninterleaved_output))

print()

decoded_output, erased_codewords = decode_output_retrieve_erased(uninterleaved_output)
print("Decoded output:   ",decoded_output)
print("Erased codeword:  ",erased_codewords,"[3]")

print()

sent = np.array([string_to_bitarray('Hello'), string_to_bitarray('World')]).flatten()
print("Sent:             ",list(sent))
restored = restore_erased(decoded_output, erased_codewords)
print("Restored:         ",list(restored))
print("Restored string:  ",bitarray_to_string(restored))

Channel input:     [-1, -1, -1, 1, 1, -1, -1, -1, -1, -1, 1, 1, 1, -1, 1, -1, 1, 1, -1, 1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, 1, -1, -1, -1, -1, -1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, -1, -1, -1, -1, 1, 1, -1, 1, 1, -1, -1, 1, 1, 1, -1, 1, 1, -1, 1, -1, 1, 1, -1, -1, -1, -1, -1, -1, 1, 1, -1, 1, 1, -1, -1, -1, -1, 1, 1, -1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1, -1, 1, 1, -1, -1, -1, -1, 1, -1, 1, 1, 1, -1, 1, -1, 1, 1, -1, 1]
Channel output:    [-1, 0, -1, 1, 0, -1, -1, 0, -1, -1, 0, 1, 1, 0, 1, -1, 0, 1, -1, 0, 1, -1, 0, 1, -1, 0, -1, 1, 0, -1, 1, 0, -1, -1, 0, -1, -1, 0, 1, 1, 0, -1, -1, 0, 1, 1, 0, -1, -1, 0, -1, 1, 0, -1, 1, 0, -1, -1, 0, 1, 1, 0, 1, 1, 0, 1, -1, 0, 1, -1, 0, -1, -1, 0, -1, 1, 0, -1, 1, 0, -1, -1, 0, -1, 1, 0, -1, 1, 0, -1, -1, 0, -1, -1, 0, -1, -1, 0, -1, 1, 0, -1, 1, 0, -1, -1, 0, -1, 1, 0, 1, 1, 0, -1, 1, 0, 1, 1, 0, 1]

Uninterleaved in:  [-1, 1, -1, -1, 1, -1, -1, -1, -1, 1, 1, -1, -1, 1, -1, 1, -1, 1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, 1, -1,